<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_mlp_regression_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch torchvision scikit-learn joblib

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import make_regression
from sklearn.preprocessing import PolynomialFeatures, QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error,
    classification_report,
    confusion_matrix,
    accuracy_score,
)
from sklearn.pipeline import Pipeline
import joblib

# ------------------------------------------------------------------------------
# 1. Reproducibility
# ------------------------------------------------------------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# ------------------------------------------------------------------------------
# 2. Generate (or load) your dataset
# ------------------------------------------------------------------------------
X, y_cont = make_regression(
    n_samples=10000, n_features=10, n_informative=8, noise=0.1, random_state=SEED
)
X_train, X_val, y_train_cont, y_val_cont = train_test_split(
    X, y_cont, test_size=0.3, random_state=SEED
)

# ------------------------------------------------------------------------------
# 3. Preprocessing pipeline
#    a) Feature expansion + quantile‐normalization
#    b) Quantile binning of the target
# ------------------------------------------------------------------------------
n_bins = 10

feature_pipeline = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=2, include_bias=False)),
        ("qt", QuantileTransformer(output_distribution="normal")),
    ]
)

# Fit/transform features, then cast to float32
X_train_proc = feature_pipeline.fit_transform(X_train).astype(np.float32)
X_val_proc = feature_pipeline.transform(X_val).astype(np.float32)

# Build a transformer to map y_cont → uniform[0,1]
qt_y = QuantileTransformer(n_quantiles=1000, output_distribution="uniform")
y_train_u = qt_y.fit_transform(y_train_cont.reshape(-1, 1)).flatten()
y_val_u = qt_y.transform(y_val_cont.reshape(-1, 1)).flatten()

# Digitize into bins 0..n_bins-1
edges = np.linspace(0, 1, n_bins + 1)


def cont_to_bin(y, edges):
    return np.digitize(y, edges[1:-1], right=True)


y_train_cls = cont_to_bin(y_train_u, edges)
y_val_cls = cont_to_bin(y_val_u, edges)

# ------------------------------------------------------------------------------
# 4. PyTorch Dataset & DataLoader
# ------------------------------------------------------------------------------
BATCH_SIZE = 128

train_ds = TensorDataset(
    torch.from_numpy(X_train_proc),
    torch.from_numpy(y_train_cont.astype(np.float32)).unsqueeze(-1),
)
val_ds = TensorDataset(
    torch.from_numpy(X_val_proc),
    torch.from_numpy(y_val_cont.astype(np.float32)).unsqueeze(-1),
)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

# ------------------------------------------------------------------------------
# 5. Model definition
# ------------------------------------------------------------------------------
class MLPRegressor(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256), nn.ReLU(),
            nn.Linear(256, 128),   nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.net(x)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPRegressor(in_dim=X_train_proc.shape[1]).to(device)

# ------------------------------------------------------------------------------
# 6. Loss, optimizer, scheduler, early stopping
# ------------------------------------------------------------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Remove the deprecated verbose flag
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=5
)


class EarlyStopping:
    def __init__(self, patience=10, min_delta=1e-5):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float("inf")
        self.counter = 0
        self.early_stop = False

    def step(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
        if self.counter >= self.patience:
            self.early_stop = True


early_stopper = EarlyStopping(patience=10)

# ------------------------------------------------------------------------------
# 7. Training loop
# ------------------------------------------------------------------------------
max_epochs = 100

for epoch in range(1, max_epochs + 1):
    # ---- Train ----
    model.train()
    train_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
    train_loss /= len(train_loader.dataset)

    # ---- Validation ----
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            val_loss += criterion(model(xb), yb).item() * xb.size(0)
    val_loss /= len(val_loader.dataset)

    scheduler.step(val_loss)
    early_stopper.step(val_loss)

    print(f"Epoch {epoch:03d} ─ Train MSE: {train_loss:.6f} ─ Val MSE: {val_loss:.6f}")
    if early_stopper.early_stop:
        print(f"Early stopping at epoch {epoch}")
        break

# ------------------------------------------------------------------------------
# 8. Final evaluation & binning
# ------------------------------------------------------------------------------
model.eval()
with torch.no_grad():
    # Cast to float32 before inference
    tX_train = torch.from_numpy(X_train_proc).to(device)
    tX_val = torch.from_numpy(X_val_proc).to(device)

    pred_train = model(tX_train).cpu().numpy().flatten()
    pred_val = model(tX_val).cpu().numpy().flatten()

mse_train = mean_squared_error(y_train_cont, pred_train)
mse_val = mean_squared_error(y_val_cont, pred_val)
print(f"\nFinal MSE ─ Train: {mse_train:.6f} ─ Val: {mse_val:.6f}")

pred_val_u = qt_y.transform(pred_val.reshape(-1, 1)).flatten()
pred_val_cls = cont_to_bin(pred_val_u, edges)

acc = accuracy_score(y_val_cls, pred_val_cls)
print(f"\nBinned Accuracy: {acc:.4f}\n")

print("Classification Report:")
print(classification_report(y_val_cls, pred_val_cls, digits=4))

print("Confusion Matrix:")
print(confusion_matrix(y_val_cls, pred_val_cls))

# ------------------------------------------------------------------------------
# 9. Save artifacts
# ------------------------------------------------------------------------------
os.makedirs("outputs", exist_ok=True)
torch.save(model.state_dict(), "outputs/mlp_regressor.pt")
joblib.dump(feature_pipeline, "outputs/feature_pipeline.pkl")
joblib.dump(qt_y, "outputs/target_transformer.pkl")

print("\nDone. Models & transformers in ./outputs")